In [2]:
from fastai.text.all import *
import joblib

path = Path('/home/harpreet/Insync/google_drive_shaannoor/Machine_Learning/2_Courses/DL_fastai/fastai_course/fastai_course/practise_Apr_2022/Week_1/multiclass_project/datasets')
df_raw = joblib.load(path/'df_raw_cleaned.joblib')

#path = rank0_first(path)

df_raw_very_small = df_raw.copy().sample(frac=0.001, random_state=42)

from sklearn.model_selection import train_test_split

train, valid = train_test_split(df_raw_very_small['text'], test_size=0.2, random_state=123)

spacy = WordTokenizer()
tokenizer = Tokenizer(spacy)

train_tokens = L([tokenizer(text) for text in train])
valid_tokens = L([tokenizer(text) for text in valid])

numericalizer = Numericalize(min_freq=5, max_vocab=60000)
numericalizer.setup(train_tokens)

def get_dataset(tokens, seq_len, bs):
    dataset = []
    #tokens = L([tokenizer(text) for text in df])
    num_tokens = tokens.map(numericalizer)
    for nums in num_tokens:
        dataset.extend(nums)
    dataset = TensorText(dataset)
    num_tokens_in_row = dataset.shape[0] // bs
    num_batches = num_tokens_in_row // (seq_len + 1)
    dataset= dataset.narrow(0, 0, num_batches*(seq_len+1)*bs)
    dataset = dataset.view(bs, -1)
    sequences =   L((dataset[:,i : i + seq_len], dataset[:, i +1 : i + seq_len + 1]) 
         for i in range(0 , seq_len*num_batches, seq_len))
    return sequences

dls = TextDataLoaders.from_dsets(get_dataset(train_tokens, seq_len=72, bs=128),
                             get_dataset(valid_tokens, seq_len=72, bs=128),
                             bs=128, drop_last=True, shuffle=False)


In [3]:
train_loader = dls.train_ds

In [4]:
for x, y in train_loader:
    print(x.shape,y.shape)
    break

torch.Size([128, 72]) torch.Size([128, 72])


In [5]:
learn = language_model_learner(dls, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()])

AttributeError: 'list' object has no attribute 'vocab'